In [1]:
import pandas as pd

## Equipos clasificados fases finales 2024

In [5]:
df_fg_24 = pd.read_csv('data/01_fase_grupos/UCL_FG_2024.csv')

# seleccionamos los equipos clasificados a las fases finales
df_clasif_24 = df_fg_24[ (df_fg_24['Rk'] == 1) | (df_fg_24['Rk'] == 2)]
df_clasif_24.set_index('Team_ID', inplace=True)
df_clasif_24

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Group,Year
Team_ID,,,,,,,,,,,,
2,1,de Bayern Munich,6,5,1,0,12,6,6,16,A,2024
59,2,dk FC Copenhagen,6,2,2,2,8,8,0,8,A,2024
5,1,eng Arsenal,6,4,1,1,16,4,12,13,B,2024
11,2,nl PSV Eindhoven,6,2,3,1,8,10,-2,9,B,2024
21,1,es Real Madrid,6,6,0,0,16,7,9,18,C,2024
87,2,it Napoli,6,3,1,2,10,9,1,10,C,2024
14,1,es Real Sociedad,6,3,3,0,7,2,5,12,D,2024
7,2,it Inter,6,3,3,0,8,5,3,12,D,2024
66,1,es Atlético Madrid,6,4,2,0,17,6,11,14,E,2024


In [9]:
equipos_clasif_24 = df_clasif_24.index.tolist()
print(f'En total hay {len(equipos_clasif_24)} equipos clasificados para las fases finales de la UCL 2023/24')

En total hay 16 equipos clasificados para las fases finales de la UCL 2023/24


In [35]:
df_clasif_24['Squad'].tolist()

['de Bayern Munich',
 'dk FC Copenhagen',
 'eng Arsenal',
 'nl PSV Eindhoven',
 'es Real Madrid',
 'it Napoli',
 'es Real Sociedad',
 'it Inter',
 'es Atlético Madrid',
 'it Lazio',
 'de Dortmund',
 'fr Paris S-G',
 'eng Manchester City',
 'de RB Leipzig',
 'es Barcelona',
 'pt Porto']

## Histórico Resultados UCL 2004 - 2023
Antes de calcular la probabilidad veamos algunas estadíticas de los equipos.

In [15]:
# dataset historico clasificacion fase grupos
df_fg = pd.read_csv('data/UCL_FG_comp_2004_2023.csv').set_index('Team_ID')

# dataset historico clasificacion fases finales
df_cf = pd.read_csv('data/UCL_CF_comp_2004_2023.csv').set_index('Team_ID')

Obtenemos cuántas veces ha ganado cada equipo clasificado actualmente.

In [17]:
# reducimos los datasets a los equipos actualmente clasificados
df_fg_interes = df_fg[df_fg.index.isin(equipos_clasif_24)]
df_cf_interes = df_cf[df_cf.index.isin(equipos_clasif_24)]

De los equipos clasificados actualmente, estos son los que han ganado alguna UCL desde el 2004

In [21]:
df_ganadores = df_cf_interes[df_cf_interes['Rk']==1]
df_num_ganadores = df_ganadores['Squad'].value_counts()
df_num_ganadores

Squad
es Real Madrid         5
es Barcelona           4
de Bayern Munich       2
pt Porto               1
it Inter               1
eng Manchester City    1
Name: count, dtype: int64

Veamos cuántas veces nuestros equipos de interés han participado en la UCL desde el 2004

In [26]:
# numero de veces que nuestros equipos de interes han participado en el campeonato
df_num_particip = df_fg_interes['Squad'].value_counts()
df_num_particip

Squad
es Real Madrid         20
de Bayern Munich       19
es Barcelona           19
pt Porto               18
eng Arsenal            14
it Inter               14
fr Paris S-G           12
es Atlético Madrid     12
eng Manchester City    12
de Dortmund            11
nl PSV Eindhoven        9
it Napoli               7
dk FC Copenhagen        5
de RB Leipzig           5
it Lazio                3
es Real Sociedad        2
Name: count, dtype: int64

In [27]:
df_num_particip.sum()

182

Veamos cuántas veces nuestros equipos de interés se han clasificado para las fases finales (octavos, cuartos, semis...) desde el 2004

In [22]:
df_clasif_final = df_fg_interes[ (df_fg_interes['Rk']==1) | (df_fg_interes['Rk']==2) ]
df_num_clasif_final = df_clasif_final['Squad'].value_counts()
df_num_clasif_final

Squad
es Real Madrid         20
de Bayern Munich       19
es Barcelona           17
eng Arsenal            14
pt Porto               13
fr Paris S-G           11
it Inter               10
eng Manchester City    10
es Atlético Madrid      9
de Dortmund             8
nl PSV Eindhoven        4
it Napoli               4
de RB Leipzig           3
es Real Sociedad        1
dk FC Copenhagen        1
it Lazio                1
Name: count, dtype: int64

In [25]:
df_num_clasif_final.sum()

145

## Cálculo de probabilidad
Vamos a obtener la probabilidad de ganar la UCL de todos los equipos actualmente clasificados para las fases finales de la UCL 2024, basándonos en los resultados históricos de la UCL desde 2004 hasta 2023.

Para tener en cuenta el histórico de resultados en tempradas anteriores, vamos a asignar puntos a los equipo en función de hasta qué fase llegaron en cada temporada. Los puntos se asignarán de la siguiente manera:
- si ha ganado la UCL: 6 puntos
- si ha llegado a la final: 5 puntos
- si ha llegado a semifinales: 4 puntos
- si ha llegado a cuartos de final: 3 puntos
- si ha llegado a octavos de final: 2 puntos
- si ha participado en la fase de grupos: 1 punto

In [40]:
stage_puntos = {
    'W': 6,
    'F': 5,
    'SF': 4,
    'QF': 3,
    'R16': 2,
    'GS': 1
}

In [38]:
# puntos de cada equipo
df_cf_interes['Points'] = df_cf_interes['Stage'].map(stage_puntos)
puntos_equipo = df_cf_interes.groupby('Squad')['Points'].sum().reset_index()

# total puntos
puntos_totales = puntos_equipo['Points'].sum()

# Calcular la probabilidad de ganar para cada equipo
puntos_equipo['Probability'] = puntos_equipo['Points'] / puntos_totales

puntos_equipo.sort_values(by='Probability', ascending=False)

C:\Users\velir\AppData\Local\Temp\ipykernel_19424\3447093208.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf_interes['Points'] = df_cf_interes['Stage'].map(stage_points)


,Squad,Points,Probability
0,de Bayern Munich,67.0,0.152968
8,es Real Madrid,67.0,0.152968
7,es Barcelona,65.0,0.148402
4,eng Arsenal,36.0,0.082192
15,pt Porto,34.0,0.077626
10,fr Paris S-G,31.0,0.070776
6,es Atlético Madrid,29.0,0.066210
5,eng Manchester City,28.0,0.063927
11,it Inter,25.0,0.057078
1,de Dortmund,22.0,0.050228


In [39]:
puntos_equipo['Probability'].sum()

1.0

Los equipos con mayor probabilidad de ganar la UCL en 2024 son:
- Bayern Munich con un 15.30%
- Real Madrid con un 15.30%
- Barcelona con un 14.84%